In [103]:
from itertools import islice

a = map(lambda x: x - 1, range(10))
slice_of_a = islice(a, 5)
slice_of_a

In [104]:
list(slice_of_a)

[-1, 0, 1, 2, 3]

In [105]:
import polars as pl

df = pl.DataFrame({"a": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})
df

a
i64
1
2
3
4
5
6
7
8
9


In [106]:
df2 = pl.DataFrame({"a": [None, None, None, None, None]}, schema={"a": pl.Null})
df2

a
null
null
null
null
null
null


In [107]:
df3 = pl.concat([df2, df], how="diagonal_relaxed")
df3

a
i64
null
null
null
null
null
1
2
3
4


In [108]:
def get_join_table(
    left: pl.LazyFrame,
    right: pl.LazyFrame,
    left_on: set[str],
    right_pkey: str,
) -> pl.LazyFrame:
    left_on = left_on.intersection(left.columns)
    left_join_table = left.select(left_on)
    joins = map(
        lambda x: left_join_table.join(
            right, left_on=x, right_on=right_pkey, how="inner"
        ),
        left_on,
    )

    left_join_table = pl.concat(joins)

    return left_join_table

In [109]:
df1 = pl.LazyFrame(
    {
        "a": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
        "aa": [11, 22, 33, 4, 5, 6, 7, 8, 9, 10, None],
        "borrow": [
            "38393939",
            "43243434",
            "barto",
            "beano",
            "super",
            "blyat",
            "blyat",
            "blyat",
            "blyat",
            "blyat",
            "blyat",
        ],
    }
)

df2 = pl.LazyFrame(
    {
        "bean": [1, 1, 2, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 22],
        "b": [
            "cat",
            "dog",
            "bean",
            "soup",
            "cat",
            "dog",
            "bean",
            "soup",
            "cat",
            "dog",
            "bean",
            "soup",
            "cat",
            "eeeeee",
            "000000",
        ],
    },
)

# df2 = df2.group_by("bean").agg("b")

df3 = df1.join(df2, left_on="a", right_on="bean", how="left")
df3 = df3.join(df2, left_on="aa", right_on="bean", how="left")
df3.collect()

a,aa,borrow,b,b_right
i64,i64,str,str,str
1,11,"""38393939""","""cat""","""eeeeee"""
1,11,"""38393939""","""dog""","""eeeeee"""
2,22,"""43243434""","""bean""","""000000"""
2,22,"""43243434""","""soup""","""000000"""
3,33,"""barto""","""cat""",null
3,33,"""barto""","""dog""",null
4,4,"""beano""","""bean""","""bean"""
5,5,"""super""","""soup""","""soup"""
6,6,"""blyat""","""cat""","""cat"""


In [110]:
join_table = get_join_table(df1, df2, {"a", "aa"}, "bean")
join_table.collect()

aa,a,b
i64,i64,str
4,4,"""bean"""
5,5,"""soup"""
6,6,"""cat"""
7,7,"""dog"""
8,8,"""bean"""
9,9,"""soup"""
10,10,"""cat"""
11,1,"""eeeeee"""
22,2,"""000000"""


In [111]:
join_table = join_table.group_by("a").agg("b")
df1 = df1.join(join_table, on="a", how="left")
df1.collect()

a,aa,borrow,b
i64,i64,str,list[str]
1,11,"""38393939""","[""eeeeee"", ""cat"", ""dog""]"
2,22,"""43243434""","[""000000"", ""bean"", ""soup""]"
3,33,"""barto""","[""cat"", ""dog""]"
4,4,"""beano""","[""bean"", ""bean""]"
5,5,"""super""","[""soup"", ""soup""]"
6,6,"""blyat""","[""cat"", ""cat""]"
7,7,"""blyat""","[""dog"", ""dog""]"
8,8,"""blyat""","[""bean"", ""bean""]"
9,9,"""blyat""","[""soup"", ""soup""]"
